In [1]:
import rpy2.robjects as robjects
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from numpy import loadtxt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pickle
import gc
import sys

/bin/bash: /home/asaf/miniconda3/envs/dan7/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2025-03-10 09:57:02.871202: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-10 09:57:02.937169: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-10 09:57:02.955704: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-10 09:57:03.243771: W tens

In [2]:
M = 100
B = 500
n_train = 1000
n_val = 200
n_test = 1000

In [3]:
r_source = robjects.r['source']
r_source("twoclasssims.R")

value,[RTYPES.CLOSXP]
visible,[RTYPES.LGLSXP]


In [4]:
simR = robjects.globalenv["sim"]

In [5]:
def sim(n):
    ret = simR(n)
    with (robjects.default_converter + pandas2ri.converter).context():
        pd_from_r_df = robjects.conversion.get_conversion().rpy2py(ret)
    return pd_from_r_df

In [6]:
(sim(1000).Class == "Class2").mean()

0.415

In [7]:
sim(1000)

,TwoFactor1,TwoFactor2,Linear01,Linear02,Linear03,Linear04,Linear05,Linear06,Linear07,Linear08,Linear09,Linear10,Nonlinear1,Nonlinear2,Nonlinear3,p,Class
1,1.167583,-0.906375,-0.235831,1.404153,0.720846,0.285401,-0.924058,1.853526,-0.376756,-0.509902,1.197951,0.103598,0.555159,0.012673,0.913378,7.576427e-10,Class1
2,0.821312,2.533838,-0.193255,1.201044,1.062711,0.326516,0.986152,0.947547,-0.612700,-0.654574,-0.285227,-0.853064,-0.219745,0.606988,0.994062,9.990737e-01,Class2
3,0.181445,0.269797,0.151392,0.372862,0.197272,2.356047,0.065067,0.382931,1.684213,-0.747575,-0.064278,1.090683,-0.094790,0.882775,0.682842,2.286942e-03,Class1
4,-0.998308,0.721973,-1.429589,-0.125264,-1.013612,-0.048453,-0.756176,0.360564,0.087375,0.738171,1.044010,1.289193,0.452807,0.225105,0.168387,3.415847e-02,Class1
5,-0.598626,-1.790068,0.925252,-1.042507,-0.275498,0.175261,0.930688,-0.286384,0.480869,-0.386264,-0.588932,-0.356419,0.709397,0.844769,0.775689,2.883260e-01,Class2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,0.482811,0.733282,-0.289784,0.655848,-1.165513,0.768492,-1.000350,-0.274521,1.344269,-0.957317,1.167325,-0.056344,0.278489,0.342425,0.125649,1.462958e-03,Class1
997,1.099120,0.338414,0.248076,0.481338,0.863230,-0.454082,-0.119846,-0.925383,-1.093575,-0.545403,-0.201346,0.228146,-0.012224,0.119182,0.148561,4.269457e-03,Class2
998,-0.255588,1.350561,-0.214890,0.240625,0.957049,-1.645766,-2.202577,-0.787895,0.338652,-1.267944,-0.041282,-0.718327,-0.135028,0.949420,0.667067,9.987892e-01,Class2
999,-0.266436,-0.050224,0.126693,-0.640259,1.652801,1.085810,-2.367181,0.213665,1.411801,1.327676,0.317156,-0.544926,-0.787141,0.236571,0.244857,8.172659e-03,Class1


In [8]:
fe = [
    'TwoFactor1',
    'TwoFactor2',
    'Linear01',
    'Linear02',
    'Linear03',
    'Linear04',
    'Linear05',
    'Linear06',
    'Linear07',
    'Linear08',
    'Linear09',
    'Linear10',
    'Nonlinear1',
    'Nonlinear2',
    'Nonlinear3']

In [9]:
def train_model(df_train,df_val):
    X_train = df_train[fe]
    y_train = df_train.Class == "Class2"
    X_val = df_val[fe]
    y_val = df_val.Class == "Class2"
    model = Sequential()
    model.add(Dense(128, input_shape=(X_train.shape[1],), activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    #callback = [keras.callbacks.EarlyStopping(monitor='val_loss',patience=1000,restore_best_weights=True)]
    callback = [keras.callbacks.EarlyStopping(monitor='val_loss',patience=8,restore_best_weights=True)]
    
    model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.01), metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=500, batch_size=500,validation_data=(X_val, y_val),callbacks=callback,verbose=0)
    return model


def get_theta(df_train,df_val,df_test):
    res = []
    for i in range(M):
        model = train_model(df_train,df_val)
        ph = model.predict(df_test[fe],verbose=0)
        ph = ph.reshape(ph.shape[0])
        res += [ph]
        del model
        keras.backend.clear_session()
        gc.collect()
    thetha = res[0]
    thetha_M = np.stack(res).mean(axis=0)
    return thetha,thetha_M,res
def get_boot(df_train,df_val,df_test):
    res_boot = []
    for i in range(B):
        df_train_boot = df_train.sample(n=df_train.shape[0],replace=True)
        df_val_boot = df_val.sample(n=df_val.shape[0],replace=True)
        #df_val = sim(n_val)
        model = train_model(df_train_boot,df_val_boot)
        ph = model.predict(df_test[fe],verbose=0)
        ph = ph.reshape(ph.shape[0])
        res_boot += [ph]
        del model
        keras.backend.clear_session()
        gc.collect()
    res_boot = np.stack(res_boot)
    return res_boot

In [10]:
df_test = sim(n_test)
thetha_arr = []
res_boot_arr = []
k=100
for i in range(k):
    print(i)
    df_train = sim(n_train)
    df_val = sim(n_val)
    thetha_arr += [get_theta(df_train,df_val,df_test)]
    res_boot_arr += [get_boot(df_train,df_val,df_test)]

0


2025-03-10 09:57:04.049472: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-03-10 09:57:04.053082: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-03-10 09:57:04.053184: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-03-10 09:57:04.053631: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compi

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [14]:
res = 0
res_width = 0
res_all = []
for i in range(k):
    thetha, thetha_M,rr = thetha_arr[i]
    res_boot = res_boot_arr[i]
    delta = np.percentile(np.abs(res_boot - thetha_M),90,axis=0)
    tmp = ((df_test.p > thetha - delta) & (df_test.p < thetha + delta)).mean()
    res += tmp
    res_all += [tmp]
    res_width  += delta.mean()

bw = (1.96*np.array(res_all).std()/10)
print(res/k,res/k - bw, res/k + bw,res_width/k)

0.8740000000000004 0.8570444252754446 0.8909555747245563 0.16312386958299335


In [13]:
res = 0
res_width = 0
res_all = []
for i in range(k):
    thetha, thetha_M,rr = thetha_arr[i]
    res_boot = res_boot_arr[i]
    delta = np.percentile(np.abs(res_boot - thetha),90,axis=0)
    tmp = ((df_test.p > thetha - delta) & (df_test.p < thetha + delta)).mean()
    res += tmp
    res_all += [tmp]
    res_width  += delta.mean()

bw = (1.96*np.array(res_all).std()/10)
print(res/k,res/k - bw, res/k + bw,res_width/k)

0.9533899999999995 0.9503457603303282 0.9564342396696708 0.19204022825196773
